In [1]:
import pandas as pd
import keras as k
import numpy as np
from keras.preprocessing.text import Tokenizer
# from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

In [3]:
data = pd.read_csv('./dataset/dataset.csv')

In [4]:
data.head()

,patent_abstract,patent_title
0,""" This invention is a novel high-speed neural ...","""Electronic neural network for solving """"trave..."
1,Systems and methods are disclosed to recognize...,3D convolutional neural networks for automatic...
2,A supervised procedure for obtaining weight va...,Accelerated training apparatus for back propag...
3,A method of accelerating the training of an ar...,Accelerating learning in neural networks
4,An apparatus is described herein. The apparatu...,Accumulator constrained quantization of convol...


In [5]:
# training_dict, word_idx, idx_word, sequences = get_data('./dataset/dataset.csv', training_len = 50)

In [6]:
data.columns

Index(['patent_abstract', 'patent_title'], dtype='object')

In [7]:
abstracts = data['patent_abstract'].values

In [8]:
abstracts[100][:300]

'Methods and apparatuses for backlash compensation. A dynamics inversion compensation scheme is designed for control of nonlinear discrete-time systems with input backlash. The techniques of this disclosure extend the dynamic inversion technique to discrete-time systems by using a filtered prediction'

In [9]:
tokenizer = Tokenizer(num_words = None, filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n', lower = False, split = ' ')
tokenizer.fit_on_texts(abstracts)
sequences = tokenizer.texts_to_sequences(abstracts)

In [10]:
sequences[100][:15]

[885, 4, 3034, 7, 3035, 6565, 17, 1287, 2604, 2605, 977, 8, 658, 7, 51]

In [11]:
# mapping indexes to words using index words
idx_word = tokenizer.index_word

' '.join(idx_word[w] for w in sequences[100][:42])

'Methods and apparatuses for backlash compensation. A dynamics inversion compensation scheme is designed for control of nonlinear discrete time systems with input backlash. The techniques of this disclosure extend the dynamic inversion technique to discrete time systems by using a filtered prediction,'

In [25]:
print(idx_word[1])

the


In [12]:
features = []
labels = []

training_length = 50

for seq in sequences:
#     create multiple training examples form each sequence
    for i in range(training_length, len(seq)):
        extract = seq[i - training_length:i + 1]
        features.append(extract[:-1])
        labels.append(extract[-1])
        
features = np.array(features)

In [18]:
features.shape

(84154, 50)

In [20]:
# One hot encoding the labels
num_words = len(idx_word) + 1
label_array = np.zeros((len(features), num_words), dtype = np.int8)

for example_index, word_index in enumerate(labels):
    label_array[example_index, word_index] = 1
    
label_array.shape

(84154, 10132)

In [21]:
print(num_words)

10132


In [41]:
glove_vectors = './dataset/glove.6B/glove.6B.100d.txt'
glove = np.loadtxt(glove_vectors, dtype='str', comments=None)

vectors = glove[:,1:].astype('float')
words = glove[:, 0]

word_lookup = {word: vector for word, vector in zip(words, vectors)}

embedding_matrix = np.zeros((num_words, vectors.shape[1]))

for i, word in enumerate(idx_word.keys()):
     vector = word_lookup.get(idx_word[word], None)

     if vector is not None:
        embedding_matrix[i + 1, :] = vector

In [ ]:
for i, word in enumerate(idx_word.keys()):
    print(word_lookup.get(idx_word[word], None))

In [55]:
word_lookup.get('parallel', None)

array([-5.8047e-02,  3.1755e-01, -1.3962e-01,  8.9352e-01,  9.1169e-01,
       -1.2790e-01, -3.6617e-01,  5.7105e-01, -2.7472e-01,  1.1765e-02,
        1.7179e-01,  2.0147e-01,  8.3605e-01, -1.4204e-01,  2.9901e-01,
        6.5877e-01, -1.3978e-01,  2.9055e-01, -5.5154e-02, -6.4442e-01,
        6.9977e-01,  3.7503e-01,  7.7342e-01, -7.8217e-02,  2.0665e-01,
       -1.1075e-01,  1.6104e-02, -5.5733e-02,  3.8644e-01,  1.1022e-01,
       -9.5521e-02,  9.0990e-02, -4.2277e-01, -8.0655e-01,  8.5612e-01,
        2.6637e-01,  3.3804e-01, -4.8534e-01,  7.0720e-02,  6.5147e-01,
        9.2254e-02, -2.5710e-01, -3.5629e-01,  3.1867e-01, -3.6363e-01,
        8.7018e-02,  5.7926e-01, -2.4927e-01,  1.4656e-03,  5.3542e-01,
        9.2430e-02,  5.2260e-01,  2.6866e-01,  7.6084e-01, -9.3371e-02,
       -1.7127e+00,  5.8050e-01,  4.7028e-01,  1.2986e+00,  4.1722e-01,
       -6.1304e-01,  4.3239e-01,  7.3693e-01,  1.7390e-01,  6.7784e-01,
       -3.1818e-02, -4.3835e-01,  2.0988e-01,  1.4622e-01,  1.23

In [53]:
idx_word[300]

'parallel'

In [54]:
embedding_matrix[300]

array([-5.8047e-02,  3.1755e-01, -1.3962e-01,  8.9352e-01,  9.1169e-01,
       -1.2790e-01, -3.6617e-01,  5.7105e-01, -2.7472e-01,  1.1765e-02,
        1.7179e-01,  2.0147e-01,  8.3605e-01, -1.4204e-01,  2.9901e-01,
        6.5877e-01, -1.3978e-01,  2.9055e-01, -5.5154e-02, -6.4442e-01,
        6.9977e-01,  3.7503e-01,  7.7342e-01, -7.8217e-02,  2.0665e-01,
       -1.1075e-01,  1.6104e-02, -5.5733e-02,  3.8644e-01,  1.1022e-01,
       -9.5521e-02,  9.0990e-02, -4.2277e-01, -8.0655e-01,  8.5612e-01,
        2.6637e-01,  3.3804e-01, -4.8534e-01,  7.0720e-02,  6.5147e-01,
        9.2254e-02, -2.5710e-01, -3.5629e-01,  3.1867e-01, -3.6363e-01,
        8.7018e-02,  5.7926e-01, -2.4927e-01,  1.4656e-03,  5.3542e-01,
        9.2430e-02,  5.2260e-01,  2.6866e-01,  7.6084e-01, -9.3371e-02,
       -1.7127e+00,  5.8050e-01,  4.7028e-01,  1.2986e+00,  4.1722e-01,
       -6.1304e-01,  4.3239e-01,  7.3693e-01,  1.7390e-01,  6.7784e-01,
       -3.1818e-02, -4.3835e-01,  2.0988e-01,  1.4622e-01,  1.23

In [52]:
embedding_matrix.shape

(10132, 100)

In [141]:


model = Sequential()

# Embedding layer
model.add(
    Embedding(input_dim=num_words,
              input_length = training_length,
              input_shape =(50,),
              output_dim=100,
              weights=[embedding_matrix],
              trainable=False,
              mask_zero=True))

# Masking layer for pre-trained embeddings
model.add(Masking(mask_value=0.0))

# Recurrent layer
model.add(LSTM(64, return_sequences=False))
#                dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
# model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='sigmoid'))

# Compile the model
model.compile(
optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [142]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 50, 100)           1013200   
_________________________________________________________________
masking_11 (Masking)         (None, 50, 100)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_12 (Dense)             (None, 10132)             658580    
Total params: 1,718,180
Trainable params: 704,980
Non-trainable params: 1,013,200
_________________________________________________________________


In [143]:
history = model.fit(features,  label_array, 
                    batch_size=2048, epochs=2)

Epoch 1/2
84154/84154 [==============================] - 105s 1ms/step - loss: 8.5384 - acc: 0.0643
Epoch 2/2
84154/84154 [==============================] - 103s 1ms/step - loss: 7.3602 - acc: 0.0216


In [171]:
test_input = []
test = np.zeros((50, 50))

import random
for x in range(50):
  test_input.append(random.randint(1,10132))
for i in range(48):
    test[0][i] = test_input[i+2]
test[0][48]=2
test[0][49]=2
print(test)

[[5.018e+03 2.877e+03 6.943e+03 ... 7.030e+02 2.000e+00 2.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]


In [172]:
output = model.predict(test,verbose=0)

In [173]:
word = output[0]


In [174]:
print(np.argmax(word))
print(output[0])

1
[3.3466634e-04 9.9915624e-01 9.9913943e-01 ... 6.1692698e-03 3.2237084e-03
 2.1323897e-03]


In [175]:
print(idx_word[2])

of
